In [1]:
using Interpolations
using Distributed
using DataFrames
using Printf
using JLD
using CSV
using Dierckx
using Dates
using PyPlot
# using PyCall
using Seaborn
using LaTeXStrings

main_path = "/home/artur/BondPricing"
module_path = string(main_path, "/", "Julia/modules/")
modls = ["Batch", "ModelObj", 
         "AnalyticFunctions", "BondPrInterp",
         "EqFinDiff", "FullInfoEq",
         "ModelPlots", "JointEqStructs", "JointEq"]
for modl in modls
    include(string(joinpath(module_path, modl), ".jl"))
end

ENV["LINES"] = 750
ENV["COLUMNS"] = 10000

┌ Info: Precompiling ModelObj [top-level]
└ @ Base loading.jl:1273
┌ Info: Precompiling AnalyticFunctions [top-level]
└ @ Base loading.jl:1273
┌ Info: Precompiling BondPrInterp [top-level]
└ @ Base loading.jl:1273
┌ Info: Precompiling EqFinDiff [top-level]
└ @ Base loading.jl:1273
┌ Info: Precompiling Batch [top-level]
└ @ Base loading.jl:1273
┌ Info: Precompiling JointEqStructs [top-level]
└ @ Base loading.jl:1273
┌ Info: Precompiling FullInfoEq [top-level]
└ @ Base loading.jl:1273


Figure(PyObject <Figure size 400x100 with 1 Axes>)

10000

In [2]:
#readdir(module_path)

In [3]:
for modl in modls
    include(string(joinpath(module_path, modl), ".jl"))
end

Figure(PyObject <Figure size 400x100 with 1 Axes>)

In [2]:
# INPUTS ###################################################
k_otc = 50 * 1e-4
k_ep = 25 * 1e-4

# Measure of Safe Firms
mu_s = .2

# Safe Firm's Risk Management Costs
st_iota = 0.0 # 2.5 * 1e-4

firm_obj_fun = :firm_value 

# Transaction Costs and Volatility Risk Parameters
pardict = Dict{Symbol,Array{Float64,1}}(:sigmal => [0.15],
                                        :m  => [1.],
                                        :gross_delta => [0.02],
                                        :kappa  => [k_ep, k_otc],
                                        :mu_b => [1.0],
                                        :xi => [1.0])

# Risk Type Parameters
iota_vec = [x for x in Batch.cvm_param_values_dict[:iota] 
            if .&(x >= st_iota, x <= 20. * 1e-4)]
lambda_vec = [.2]
sigmah_vec = Batch.svm_param_values_dict[:sigmah]

# Load Unconstrained Results
cvmdf, svmdf = JointEqStructs.get_joint_types_dfs(pardict, iota_vec, 
                                           lambda_vec, sigmah_vec;
                                           firm_obj_fun=firm_obj_fun)


all_typesdict = Dict{Symbol, Array{Float64, N} where N}(:st_iota => [st_iota],
                                                        :st_lambda => [NaN],
                                                        :st_sigmah => [NaN],
                                                        :rt_iota => iota_vec,
                                                        :rt_lambda => lambda_vec,
                                                        :rt_sigmah => sigmah_vec)
tcdf = FullInfoEq.get_types_comb_df(all_typesdict)

# My choice
tc_pair = tcdf[.&(abs.(tcdf[:, :rt_iota] .- iota_vec[5]) .<1e-5, 
                  abs.(tcdf[:, :rt_sigmah] .- sigmah_vec[2]) .<1e-5), :pair_num][1]

# Form types dictionary            
tmp = tcdf[abs.(tcdf[:, :pair_num] .- tc_pair) .< 1e-5, :]
typesdict = Dict{Symbol, Float64}([x=> tmp[:, x][1] for x in names(tmp)])
# # #########################################################
            
jfotc = JointEqStructs.form_joint_types(k_otc, mu_s, pardict, typesdict;
                                        cvmdf=cvmdf, svmdf=svmdf,
                                        firm_obj_fun=firm_obj_fun,
                                        bc=nothing)
 
jfep = JointEqStructs.form_joint_types(k_ep, mu_s, pardict, typesdict;
                                       cvmdf=cvmdf, svmdf=svmdf,
                                       firm_obj_fun=firm_obj_fun,
                                       bc=nothing)

Setting parameter dictionary using (m, m_comb_num) ID pair...
Loading optimal results dataframe...
Setting parameter dictionary using (m, m_comb_num) ID pair...
Setting parameter dictionary using (m, m_comb_num) ID pair...
Loading optimal results dataframe...
Setting parameter dictionary using unique combination ID...
Setting initial vbl value to NaN
Setting initial vbh value to NaN
Setting initial c value to NaN
Setting initial p value to NaN
Constant Volatility Model: setting vbh to vbl, lambda to NaN
Setting parameter dictionary using unique combination ID...
Setting initial vbl value to NaN
Setting initial vbh value to NaN
Setting initial c value to NaN
Setting initial p value to NaN
Constant Volatility Model: setting vbh to vbl, lambda to NaN
Setting parameter dictionary using unique combination ID...
Loading SVM object...
Batch object file not found.
Batch Object is incompatible! Loading surfaces only instead.
Setting initial vbl value to NaN
Setting initial vbh value to NaN
Sett

Main.JointEqStructs.JointFirms(0.0025, Main.JointEqStructs.TypesDist(0.2, 0.0, NaN, NaN, 0.0002, 0.2, 0.225), Main.JointEqStructs.TypesCommonParams(100.0, 0.6, 0.27, 0.08, 0.02, 1.0, 0.15), Main.JointEqStructs.FirmType(Main.JointEqStructs.RMPCFirmObj(ModelObj.Firm
  mu_b: Float64 1.0
  m: Float64 1.0
  c: Float64 NaN
  p: Float64 NaN
  vbl: Float64 NaN
  vbh: Float64 NaN
  pm: ModelObj.FirmParams
  model: String "cvm"
  bi: ModelObj.BPrInputs
  bs: ModelObj.BPrSurfs
  bf: ModelObj.BPrInterpFuns
  bit: ModelObj.BPrFixedTTMInputs
  bft: ModelObj.BPrInterpFuns
  optKS: ModelObj.KStruct
, Batch.BatchStruct
  model: String "cvm"
  dfn: Batch.DirFileNames
  dfc: Batch.BatchDFColumns
  mi: Batch.ModelInputs
  bp: Batch.BatchParams
  coupon_grid: Array{Float64}((52,)) [0.25, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5  …  21.0, 21.5, 22.0, 22.5, 23.0, 23.5, 24.0, 24.5, 25.0, 25.5]
  pvb_tol_vec: Array{Float64}((0,)) Float64[]
), Main.JointEqStructs.RMPCFirmObj(nothing, nothing)), Main.JointEqS

In [3]:
# Make results directories
jks_fpath = JointEq.make_jeq_jks_fpath(jfep)

# Full Information Equilibrium
fi_fpath_name = JointEq.get_jeq_contour_fname(jks_fpath, 1; eq_type="full_info")


tmp = CSV.read(fi_fpath_name)

,gross_delta,delta,iota,kappa,lambda,sigmah,sigmal,mu_b,m,c,p,vb,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,debt,equity,firm_value,leverage,MBR,V0,r,alpha,pi,xi,eq_type,datetime,type,rmp
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,DateTime,String,String
1,0.02,0.02,0.0,0.0025,NaN,NaN,0.15,1.04516,1.0,4.5,54.0,59.8202,0.000226196,0.000226196,0.000284745,0,0.0,56.4612,55.5114,111.973,50.4241,27.4987,100.0,0.08,0.6,0.27,1.0,full_info,2019-12-22T07:32:35.434,st,rm
2,0.02,0.0198,0.0002,0.0025,NaN,NaN,0.15,1.04128,1.0,4.5,54.0,59.9514,0.000222455,0.000222455,0.000279543,0,0.0,56.2514,54.6765,110.928,50.7099,24.9788,100.0,0.08,0.6,0.27,1.0,full_info,2019-12-22T07:32:37.722,rt,rm
3,0.02,0.02,0.0,0.0025,0.2,0.225,0.15,0.854851,1.0,4.5,54.0,50.884,1.09135e-5,1.09135e-5,0.0011387,0,0.0,46.1807,62.0365,108.217,42.6741,15.2682,100.0,0.08,0.6,0.27,1.0,full_info,2019-12-22T07:34:07.151,rt,nrm


In [4]:
# Compute Full Information Solution
# fidf = FullInfoEq.get_fi_results(jfep, fi_fpath_name;
#                                   rerun_full_info=true)
# _, fieqdf = FullInfoEq.get_fi_eq(jfep; fidf=fidf)

fidf, fieqdf = FullInfoEq.get_fi_eq(jfep)

fieqdf

COUNT: 0
COUNT: 1
Computing Equity Vmax
Vmax: 125.0
phi0: -20.261288992693682
phi1: 0.9999999999999998
pv_rfdebt: 27.010946902005966
 mu_b = 0.5
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.5
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Equity Vmax: 131.25
 
eq_max: 110.98513183958794
eq_vbl: 0.0
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 79.72979801247337
Equity Core Function Computation Time: 1.3840689659118652
Total Equity FD Core Function Computation Time: 2.1991679668426514
 mu_b = 0.5
 m = 1.0
 c = 4.5
 p = 54.0
Total computation time: 3.8885929584503174
Computing Equity Vmax
Vmax: 125.0
phi0: -23.10753802218709
phi1: 0.9999999999999998
pv_rfdebt: 30.805368936717365
 mu_b = 0.5702385971228133
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.5702385971228133
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Equity Vmax: 131.25
 
eq_max: 108.13393684044262
eq_vbl: 0.0
Shape of Gamma matrix: (1498,)
Computing equity v

Computing equity values... 
V0: 100.0; equity: 20.70066778414578
Equity Core Function Computation Time: 0.02857685089111328
Total Equity FD Core Function Computation Time: 0.06367206573486328
 mu_b = 1.4833403597193866
 m = 1.0
 c = 4.5
 p = 54.0
Total computation time: 0.06958580017089844
Computing Equity Vmax
Vmax: 125.0
phi0: -62.9550244350948
phi1: 0.9999999999999998
pv_rfdebt: 83.92727742267697
 mu_b = 1.5535789568422
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 1.5535789568422
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Equity Vmax: 131.25
 
eq_max: 61.93811704057114
eq_vbl: 0.0
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 13.208655918487139
Equity Core Function Computation Time: 0.028414011001586914
Total Equity FD Core Function Computation Time: 0.05783390998840332
 mu_b = 1.5535789568422
 m = 1.0
 c = 4.5
 p = 54.0
Total computation time: 0.06371402740478516
Computing Equity Vmax
Vmax: 125.0
phi0: -65.8012734645882
phi1: 0.99

V0: 100.0; equity: 60.87291874196755
Equity Core Function Computation Time: 0.03701305389404297
Total Equity FD Core Function Computation Time: 0.0690150260925293
 mu_b = 0.9180210223154863
 m = 1.0
 c = 4.5
 p = 54.0
Total computation time: 0.07502388954162598
Computing Equity Vmax
Vmax: 125.0
phi0: -40.02379338172277
phi1: 0.9899999999999999
pv_rfdebt: 53.35694872327295
 mu_b = 0.9876911927014007
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.9876911927014007
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Equity Vmax: 131.25
 
eq_max: 89.58640464615465
eq_vbl: 0.0
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 57.47452100665728
Equity Core Function Computation Time: 0.036576032638549805
Total Equity FD Core Function Computation Time: 0.06847691535949707
 mu_b = 0.9876911927014007
 m = 1.0
 c = 4.5
 p = 54.0
Total computation time: 0.07445716857910156
Computing Equity Vmax
Vmax: 125.0
phi0: -42.84700829444121
phi1: 0.9899999999999999
pv_r

Computing equity values... 
V0: 100.0; equity: 54.67645664720925
Equity Core Function Computation Time: 0.036894798278808594
Total Equity FD Core Function Computation Time: 0.06908798217773438
 mu_b = 1.0412799335445428
 m = 1.0
 c = 4.5
 p = 54.0
Total computation time: 0.07505512237548828
COUNT: 0
Computing Equity Vmax
Vmax: 125.0
phi0: -20.261288992693682
phi1: 0.9999999999999998
pv_rfdebt: 27.010946902005966
 mu_b = 0.5
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.5
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.5
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.5
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.5
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.5
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.5
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.5
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.5
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: 0.0008255911014374627
eq_abs_per_diff: 0.0009258340689315004
Equity Vmax: 184.6819304736329
 
eq_max: 164.42011330616214
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant 

Computing equity values... 
V0: 100.0; equity: 79.3006374508324
Equity Core Function Computation Time: 0.036073923110961914
Total Equity FD Core Function Computation Time: 0.07765793800354004
Total computation time: 0.09598803520202637
Computing Equity Vmax
Vmax: 125.0
phi0: -20.261288992693682
phi1: 0.9999999999999998
pv_rfdebt: 27.010946902005966
 mu_b = 0.5
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.5
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.5
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.5
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.5
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.5
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.5
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.5
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.5
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: 0.0008255911014374627
eq_abs_per_diff: 0.0009258340689315004
Equity Vmax: 184.6819304736329
 
eq_max: 164.42011330616214
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equi

...

Computing equity values... 
V0: 100.0; equity: 0.0
Equity Core Function Computation Time: 0.033622026443481445
Total Equity FD Core Function Computation Time: 0.0753481388092041
Total computation time: 0.1463160514831543
Computing Equity Vmax
Vmax: 125.0
phi0: -135.43072116168935
phi1: 0.9999999999999998
pv_rfdebt: 180.54685560814514
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_

Computing equity values... 
V0: 100.0; equity: 0.0
Equity Core Function Computation Time: 0.033453941345214844
Total Equity FD Core Function Computation Time: 0.07519006729125977
Total computation time: 0.14627408981323242
Computing Equity Vmax
Vmax: 125.0
phi0: -135.43072116168935
phi1: 0.9999999999999998
pv_rfdebt: 180.54685560814514
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 m

Excessive output truncated after 524291 bytes.

V0: 100.0; equity: 0.0
Equity Core Function Computation Time: 0.03596782684326172
Total Equity FD Core Function Computation Time: 0.0774390697479248
Total computation time: 0.14896011352539062
Computing Equity Vmax
Vmax: 125.0
phi0: -135.43072116168935
phi1: 0.9999999999999998
pv_rfdebt: 180.54685560814514
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 3.3421052631578947
 m = 

,gross_delta,delta,iota,kappa,lambda,sigmah,sigmal,mu_b,m,c,p,vb,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,debt,equity,firm_value,leverage,MBR,V0,r,alpha,pi,xi,eq_type,datetime,type,rmp
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,DateTime,Symbol,Symbol
1,0.02,0.02,0.0,0.0025,NaN,NaN,0.15,1.04516,1.0,4.5,54.0,59.8202,0.000226196,0.000226196,0.000284745,0,0.0,56.4612,55.5114,111.973,50.4241,27.4987,100.0,0.08,0.6,0.27,1.0,full_info,2020-01-02T11:29:58.401,st,rm
2,0.02,0.0198,0.0002,0.0025,NaN,NaN,0.15,1.04128,1.0,4.5,54.0,59.9514,0.000222455,0.000222455,0.000279543,0,0.0,56.2514,54.6765,110.928,50.7099,24.9788,100.0,0.08,0.6,0.27,1.0,full_info,2020-01-02T11:30:00.606,rt,rm


In [5]:
resd = Dict{Symbol,DataFrame}(:fi => fidf, :fieq => fieqdf)

Dict{Symbol,DataFrame} with 2 entries:
  :fi   => 3×31 DataFrame…
  :fieq => 2×31 DataFrame…

In [6]:
sf, rf, jks = JointEqStructs.form_firms_jks(jfep, resd[:fieq])

(ModelObj.Firm
  mu_b: Float64 1.0
  m: Float64 1.0
  c: Float64 NaN
  p: Float64 NaN
  vbl: Float64 NaN
  vbh: Float64 NaN
  pm: ModelObj.FirmParams
  model: String "cvm"
  bi: ModelObj.BPrInputs
  bs: ModelObj.BPrSurfs
  bf: ModelObj.BPrInterpFuns
  bit: ModelObj.BPrFixedTTMInputs
  bft: ModelObj.BPrInterpFuns
  optKS: ModelObj.KStruct
, ModelObj.Firm
  mu_b: Float64 1.0
  m: Float64 1.0
  c: Float64 NaN
  p: Float64 NaN
  vbl: Float64 NaN
  vbh: Float64 NaN
  pm: ModelObj.FirmParams
  model: String "cvm"
  bi: ModelObj.BPrInputs
  bs: ModelObj.BPrSurfs
  bf: ModelObj.BPrInterpFuns
  bit: ModelObj.BPrFixedTTMInputs
  bft: ModelObj.BPrInterpFuns
  optKS: ModelObj.KStruct
, Main.JointEqStructs.JointKStruct(0.2, NaN, 1.0, 4.5, 54.0, NaN, 59.82018174178425, NaN, 59.95140013116282, NaN))

In [ ]:
# jfep = JointEqStructs.form_joint_types(k_ep, mu_s, pardict, typesdict;
#                                        cvmdf=cvmdf, svmdf=svmdf,
#                                        firm_obj_fun=firm_obj_fun,
#                                        bc=nothing)

# fidf, fieqdf = FullInfoEq.get_fi_eq(jfep)
# resd = Dict{Symbol,DataFrame}(:fi => fidf, :fieq => fieqdf)
# fisf, firf, jks = JointEqStructs.form_firms_jks(jfep, resd[:fieq])

In [13]:
for modl in modls
    include(string(joinpath(module_path, modl), ".jl"))
end

Figure(PyObject <Figure size 400x100 with 1 Axes>)

In [18]:
?FullInfoEq.get_fi_eq(jf)

No documentation found.

`Main.FullInfoEq.get_fi_eq` is a `Function`.

```
# 1 method for generic function "get_fi_eq":
[1] get_fi_eq(jf; fidf) in Main.FullInfoEq at /home/artur/BondPricing/Julia/modules/FullInfoEq.jl:433
```


In [19]:
fieqdf

,gross_delta,delta,iota,kappa,lambda,sigmah,sigmal,mu_b,m,c,p,vb,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,debt,equity,firm_value,leverage,MBR,V0,r,alpha,pi,xi,eq_type,datetime,type,rmp
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,DateTime,Symbol,Symbol
1,0.02,0.02,0.0,0.0025,NaN,NaN,0.15,1.04516,1.0,4.5,54.0,59.8202,0.000226196,0.000226196,0.000284745,0,0.0,56.4612,55.5114,111.973,50.4241,27.4987,100.0,0.08,0.6,0.27,1.0,full_info,2020-01-02T11:29:58.401,st,rm
2,0.02,0.0198,0.0002,0.0025,NaN,NaN,0.15,1.04128,1.0,4.5,54.0,59.9514,0.000222455,0.000222455,0.000279543,0,0.0,56.2514,54.6765,110.928,50.7099,24.9788,100.0,0.08,0.6,0.27,1.0,full_info,2020-01-02T11:30:00.606,rt,rm


In [ ]:
function compute_misrep(jf, fidf::DataFrame; 
                        sf_obj_fun::Symbol=:firm_value
                        rf_obj_fun::Symbol=:firm_value)
    
    # Compute Full Information Results
    fidf, fieqdf = FullInfoEq.get_fi_eq(jf; fidf)  
    fisf, firf, jks = JointEqStructs.form_firms_jks(jf, fieqdf)
    
    # Extract Full Information Info
    fi_sf_mu_b = fieqdf[fieqdf[:, :type] .== :st, :mu_b][1]
    fi_rf_mu_b = fieqdf[fieqdf[:, :type] .== :rt, :mu_b][1]
    fi_sf_obj_val = fieqdf[fieqdf[:, :type] .== :st, sf_obj_fun][1]
    fi_rf_obj_val = fieqdf[fieqdf[:, :type] .== :st, sf_obj_fun][1]
    
    # Misrepresentation Capital Structure
    misrep_jks = deepcopy(jks)
    
    # Case I: Risky Type Copies Safe Type's Capital Structure
    if !isnan(fi_sf_mu_b)
        setfield!(misrep_jks, :mu_b, fi_sf_mu_b)
        setfield!(misrep_jks, :mu_s, 1.)
    end
    
    ep_misrep_eqdf = DataFrame()
    st_rmp = fieqdf[fieqdf[:, :type] .== :st, :rmp][1]
    for rt_rmp in [:rm, :nrm]
        rf = getfield(jfep.rt, rt_rmp).fr
        if !isnothing(rf)
            misrepdf = JointEq.find_joint_optimal_vb(fisf, rf, 
                                                     misrep_jks;
                                                     mu_b=fi_sf_mu_b,
                                                     rerun_fi_vb=true)    
            misrepdf[!, :eq_type] .= :misrep
            misrepdf[!, :datetime] .= Dates.now()
            misrepdf[!, :type] = [:st, :rt]
            misrepdf[!, :rmp] = [st_rmp, rt_rmp]
            misrepdf[!, :direction] .= :rt2st
        end
        
        ep_misrep_eqdf = vcat([ep_misrep_eqdf, misrepdf]...)
    end
    
    return ep_misrep_eqdf
end

In [22]:
 isnothing(getfield(jfep.st, :nrm).fr)

true

In [17]:
resd[:fieq]

,gross_delta,delta,iota,kappa,lambda,sigmah,sigmal,mu_b,m,c,p,vb,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,debt,equity,firm_value,leverage,MBR,V0,r,alpha,pi,xi,eq_type,datetime,type,rmp
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,DateTime,Symbol,Symbol
1,0.02,0.02,0.0,0.0025,NaN,NaN,0.15,1.04516,1.0,4.5,54.0,59.8202,0.000226196,0.000226196,0.000284745,0,0.0,56.4612,55.5114,111.973,50.4241,27.4987,100.0,0.08,0.6,0.27,1.0,full_info,2020-01-02T11:29:58.401,st,rm
2,0.02,0.0198,0.0002,0.0025,NaN,NaN,0.15,1.04128,1.0,4.5,54.0,59.9514,0.000222455,0.000222455,0.000279543,0,0.0,56.2514,54.6765,110.928,50.7099,24.9788,100.0,0.08,0.6,0.27,1.0,full_info,2020-01-02T11:30:00.606,rt,rm


In [16]:
fieqdf

,gross_delta,delta,iota,kappa,lambda,sigmah,sigmal,mu_b,m,c,p,vb,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,debt,equity,firm_value,leverage,MBR,V0,r,alpha,pi,xi,eq_type,datetime,type,rmp
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,DateTime,Symbol,Symbol
1,0.02,0.02,0.0,0.0025,NaN,NaN,0.15,1.04516,1.0,4.5,54.0,59.8202,0.000226196,0.000226196,0.000284745,0,0.0,56.4612,55.5114,111.973,50.4241,27.4987,100.0,0.08,0.6,0.27,1.0,full_info,2020-01-02T11:29:58.401,st,rm
2,0.02,0.0198,0.0002,0.0025,NaN,NaN,0.15,1.04128,1.0,4.5,54.0,59.9514,0.000222455,0.000222455,0.000279543,0,0.0,56.2514,54.6765,110.928,50.7099,24.9788,100.0,0.08,0.6,0.27,1.0,full_info,2020-01-02T11:30:00.606,rt,rm


In [14]:

fisf, firf, jks = JointEqStructs.form_firms_jks(jfep, resd[:fieq])
ep_jks = deepcopy(jks)
fi_sf_mu_b = fieqdf[fieqdf[:, :type] .== :st, :mu_b][1]
fi_rf_mu_b = fieqdf[fieqdf[:, :type] .== :rt, :mu_b][1]
fi_sf_obj_val = fieqdf[fieqdf[:, :type] .== :st, sf_obj_fun][1]
fi_rf_obj_val = fieqdf[fieqdf[:, :type] .== :st, sf_obj_fun][1]

# Misrepresentation
misrep_jks = deepcopy(ep_jks)
setfield!(misrep_jks, :mu_s, 1.)

# Case I: Risky Type Copies Safe Type's Capital Structure
if !isnan(fi_sf_mu_b)
    setfield!(misrep_jks, :mu_b, fi_sf_mu_b)
end

st_rmp = :rm
rt_rmp = :rm
rf = getfield(jfep.rt, rt_rmp).fr

if !isnothing(rf)
    ep_misrep_eqdf = JointEq.find_joint_optimal_vb(fisf, rf, misrep_jks;
                                            mu_b=fi_sf_mu_b,
                                            rerun_fi_vb=true)    
    ep_misrep_eqdf[!, :eq_type] .= :misrep
    ep_misrep_eqdf[!, :datetime] .= Dates.now()
    ep_misrep_eqdf[!, :type] = [:st, :rt]
    ep_misrep_eqdf[!, :rmp] = [st_rmp, rt_rmp]
    ep_misrep_eqdf[!, :direction] .= :rt2st
end

  
vbl: 53.83816356760583
  
Computing Equity Vmax
Vmax: 125.0
phi0: -42.35267174893433
phi1: 0.9999999999999998
pv_rfdebt: 56.46164803142979
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Vmax: 125.0
phi0: -42.35267174893433
phi1: 0.9899999999999999
pv_rfdebt: 56.46164803142979
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Equity Vmax: 131.25
 
eq_max: 0.8911213257882025
eq_vbl: 0.0
eq_vbl: 0.0
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 56.38146230236786
Equity Core Function Computation Time: 0.041712045669555664
Total Equity FD Core Function Computation Time: 0.13654184341430664
Shape of Gamma matrix: (1498,)
Computing equity valu

V0: 100.0; equity: 55.869876595969906
Equity Core Function Computation Time: 0.029942035675048828
Total Equity FD Core Function Computation Time: 0.06309795379638672
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 54.88561496912277
Equity Core Function Computation Time: 0.03924393653869629
Total Equity FD Core Function Computation Time: 0.0733938217163086
Total computation time: 0.15007686614990234
  
vbl: 58.389731870173584
  
Computing Equity Vmax
Vmax: 125.0
phi0: -42.35267174893433
phi1: 0.9999999999999998
pv_rfdebt: 56.46164803142979
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Vmax: 125.0
phi0: -42.35267174893433
phi1: 0.9899999999999999
pv_rfdebt: 56.46164803142979
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Equity Vmax: 131.25
 
eq_m

Computing equity values... 
V0: 100.0; equity: 54.31333092515817
Equity Core Function Computation Time: 0.03828716278076172
Total Equity FD Core Function Computation Time: 0.07245898246765137
Total computation time: 0.23779892921447754
  
vbl: 61.6408520862934
  
Computing Equity Vmax
Vmax: 125.0
phi0: -42.35267174893433
phi1: 0.9999999999999998
pv_rfdebt: 56.46164803142979
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Vmax: 125.0
phi0: -42.35267174893433
phi1: 0.9899999999999999
pv_rfdebt: 56.46164803142979
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Equity Vmax: 131.25
 
eq_max: 0.7557790675220244
eq_vbl: 0.0
Computing Equity Vmax
Vmax: 125.0
phi0: -42.35267174893433
phi1: 0.9899999999999999
pv_rfdebt: 56.46164803142979
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 

V0: 64.24174825918927; equity: 3.3267536784619884
Equity Core Function Computation Time: 0.04336214065551758
Total Equity FD Core Function Computation Time: 0.07046890258789062
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
Total computation time: 0.07634186744689941
eq_vbl: 3.3267536784619884
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 54.49351679278341
Equity Core Function Computation Time: 0.029176950454711914
Total Equity FD Core Function Computation Time: 0.06263899803161621
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 53.77021524332207
Equity Core Function Computation Time: 0.03944206237792969
Total Equity FD Core Function Computation Time: 0.0794520378112793
Total computation time: 0.23209905624389648
  
vbl: 64.89197230241322
  
Computing Equity Vmax
Vmax: 125.0
phi0: -42.35267174893433
phi1: 0.9

V0: 100.0; equity: 56.30885637485793
Equity Core Function Computation Time: 0.036309003829956055
Total Equity FD Core Function Computation Time: 0.06968402862548828
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 55.31735068192992
Equity Core Function Computation Time: 0.04321002960205078
Total Equity FD Core Function Computation Time: 0.07023096084594727
Total computation time: 0.15355896949768066
  
vbl: 55.13861165405376
  
Computing Equity Vmax
Vmax: 125.0
phi0: -42.35267174893433
phi1: 0.9999999999999998
pv_rfdebt: 56.46164803142979
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Vmax: 125.0
phi0: -42.35267174893433
phi1: 0.9899999999999999
pv_rfdebt: 56.46164803142979
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Equity Vmax: 131.25
 
eq_ma

...

V0: 62.291076129517364; equity: 1.4161667580539012
Equity Core Function Computation Time: 0.029239177703857422
Total Equity FD Core Function Computation Time: 0.062280893325805664
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
Total computation time: 0.06820797920227051
eq_vbl: 1.4161667580539012
eq_vbl: 0.0
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 55.51137317228806
Equity Core Function Computation Time: 0.038926124572753906
Total Equity FD Core Function Computation Time: 0.07961106300354004
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 54.457033998115016
Equity Core Function Computation Time: 0.029230117797851562
Total Equity FD Core Function Computation Time: 0.06905388832092285
Total computation time: 0.2306499481201172
  
vbl: 62.94130017274133
  
Computing Equity Vmax
Vmax: 125.0
phi0: -42.3526717

Computing equity values... 
V0: 100.0; equity: 54.18424082814779
Equity Core Function Computation Time: 0.04339480400085449
Total Equity FD Core Function Computation Time: 0.0705270767211914
Total computation time: 0.22899103164672852
  
vbl: 65.5421963456372
  
Computing Equity Vmax
Vmax: 125.0
phi0: -42.35267174893433
phi1: 0.9999999999999998
pv_rfdebt: 56.46164803142979
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Vmax: 125.0
phi0: -42.35267174893433
phi1: 0.9899999999999999
pv_rfdebt: 56.46164803142979
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Equity Vmax: 131.25
 
eq_max: 0.6944097472331585
Computing Equity Vmax
Vmax: 125.0
phi0: -42.35267174893433
phi1: 0.9999999999999998
pv_rfdebt: 56.46164803142979
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 

Computing equity values... 
V0: 100.0; equity: 55.52208840669461
Equity Core Function Computation Time: 0.039192914962768555
Total Equity FD Core Function Computation Time: 0.0799551010131836
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 54.54365867723829
Equity Core Function Computation Time: 0.029690027236938477
Total Equity FD Core Function Computation Time: 0.06941986083984375
Total computation time: 0.16305112838745117
  
vbl: 59.92108982794992
  
Computing Equity Vmax
Vmax: 125.0
phi0: -42.35267174893433
phi1: 0.9999999999999998
pv_rfdebt: 56.46164803142979
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Vmax: 125.0
phi0: -42.35267174893433
phi1: 0.9899999999999999
pv_rfdebt: 56.46164803142979
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN

V0: 100.0; equity: 54.533288523955775
Equity Core Function Computation Time: 0.04327797889709473
Total Equity FD Core Function Computation Time: 0.07025408744812012
Total computation time: 0.22855305671691895
  
vbl: 60.919774658415754
  
Computing Equity Vmax
Vmax: 125.0
phi0: -42.35267174893433
phi1: 0.9999999999999998
pv_rfdebt: 56.46164803142979
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Vmax: 125.0
phi0: -42.35267174893433
phi1: 0.9899999999999999
pv_rfdebt: 56.46164803142979
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Equity Vmax: 131.25
 
eq_max: 0.7675460724270605
Computing Equity Vmax
Vmax: 125.0
phi0: -42.35267174893433
phi1: 0.9999999999999998
pv_rfdebt: 56.46164803142979
 mu_b = 1.0451623231919476
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 1.0451623231919476
 m = 

2-element Array{Symbol,1}:
 :rt2st
 :rt2st

In [15]:
ep_misrep_eqdf

,sf_defaults_first,fi_vb,st_vb,rt_vb,vb,eq_deriv,eq_min_val,mu_s,m,mu_b,c,p,eq_deriv_min_val,eq_negative,eq_vb,MBR,debt,equity,firm_value,leverage,iota,lambda,sigmah,gross_delta,delta,kappa,sigmal,V0,xi,r,alpha,pi,eq_type,datetime,type,rmp,direction
,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,DateTime,Symbol,Symbol,Symbol
1,0,59.8202,59.8202,NaN,59.9492,0.0709412,0.00718398,1.0,1.0,1.04516,4.5,54.0,0.0709412,0,0.00469109,27.4985,56.4612,55.5114,111.973,50.4241,0.0,NaN,NaN,0.02,0.02,0.0025,0.15,100.0,1.0,0.08,0.6,0.27,misrep,2020-01-02T11:50:32.529,st,rm,rt2st
2,0,60.1749,NaN,59.9492,59.9492,0.00461953,0.000416951,1.0,1.0,1.04516,4.5,54.0,0.00461953,0,0.0,25.2521,56.4612,54.5333,110.994,50.8685,0.0002,NaN,NaN,0.02,0.0198,0.0025,0.15,100.0,1.0,0.08,0.6,0.27,misrep,2020-01-02T11:50:32.529,rt,rm,rt2st


In [10]:
#
st_rmp = :rm
rt_rmp = :rm
ep_misrep_eqdf[!, :eq_type] .= :misrep
ep_misrep_eqdf[!, :datetime] .= Dates.now()
ep_misrep_eqdf[!, :type] = [:st, :rt]
ep_misrep_eqdf[!, :rmp] = [st_rmp, rt_rmp]
ep_misrep_eqdf[!, :direction] .= :rt2st
ep_misrep_eqdf

,sf_defaults_first,fi_vb,st_vb,rt_vb,vb,eq_deriv,eq_min_val,mu_s,m,mu_b,c,p,eq_deriv_min_val,eq_negative,eq_vb,MBR,debt,equity,firm_value,leverage,iota,lambda,sigmah,gross_delta,delta,kappa,sigmal,V0,xi,r,alpha,pi,eq_type,datetime,type,rmp,direction
,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,DateTime,Symbol,Symbol,Symbol
1,0,59.8202,59.8202,NaN,59.9492,0.0709412,0.00718398,1.0,1.0,1.04516,4.5,54.0,0.0709412,0,0.00469109,27.4985,56.4612,55.5114,111.973,50.4241,0.0,NaN,NaN,0.02,0.02,0.0025,0.15,100.0,1.0,0.08,0.6,0.27,misrep,2020-01-02T11:46:40.042,st,rm,rt2st
2,0,60.1749,NaN,59.9492,59.9492,0.00461953,0.000416951,1.0,1.0,1.04516,4.5,54.0,0.00461953,0,0.0,25.2521,56.4612,54.5333,110.994,50.8685,0.0002,NaN,NaN,0.02,0.0198,0.0025,0.15,100.0,1.0,0.08,0.6,0.27,misrep,2020-01-02T11:46:40.042,rt,rm,rt2st


In [17]:
ep_misrep_eqdf

,sf_defaults_first,fi_vb,st_vb,rt_vb,vb,eq_deriv,eq_min_val,mu_s,m,mu_b,c,p,eq_deriv_min_val,eq_negative,eq_vb,MBR,debt,equity,firm_value,leverage,iota,lambda,sigmah,gross_delta,delta,kappa,sigmal,V0,xi,r,alpha,pi,type
,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol
1,0,59.8202,59.8202,NaN,59.9492,0.0709412,0.00718398,1.0,1.0,1.04516,4.5,54.0,0.0709412,0,0.00469109,27.4985,56.4612,55.5114,111.973,50.4241,0.0,NaN,NaN,0.02,0.02,0.0025,0.15,100.0,1.0,0.08,0.6,0.27,st
2,0,60.1749,NaN,59.9492,59.9492,0.00461953,0.000416951,1.0,1.0,1.04516,4.5,54.0,0.00461953,0,0.0,25.2521,56.4612,54.5333,110.994,50.8685,0.0002,NaN,NaN,0.02,0.0198,0.0025,0.15,100.0,1.0,0.08,0.6,0.27,rt


In [15]:
fisf.bs

ModelObj.BPrSurfs
  vtgrid: StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}
  ttmgrid: StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}
  vbhlgrid: StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}
  f11_surf: Array{Float64}((1, 1)) [NaN]
  f12_surf: Array{Float64}((1, 1, 1)) [NaN]
  f13_surf: Array{Float64}((1, 1, 1)) [NaN]
  f21_surf: Array{Float64}((1, 1)) [NaN]
  f22_surf: Array{Float64}((1, 1)) [NaN]


In [ ]:
jks.

In [ ]:
.= string(Dates.unix2datetime(stat(batch_obj_file).mtime))

In [10]:
DateTime

DateTime

In [ ]:
FullInfoEq.get_empty_fidf()

In [ ]:
[x for x in names(fieqdf) if !(x in names(fidf))]

In [ ]:
[x for x in names(fieqdf) if !(x in names(ep_misrep_eqdf))]

In [ ]:
[x for x in names(ep_misrep_eqdf) if !(x in names(fieqdf))]

In [ ]:
jks2 = JointEq.set_full_information_vb!(sf, rf, jks)
setfield!(jks2, :mu_s, 1.)

In [ ]:
jks2 = JointEq.set_full_information_vb!(sf, rf, jks)
setfield!(jks2, :mu_s, 1.)
df = JointEq.compile_opt_vb_results(sf, rf, jks2, sfdf, rfdf)

In [ ]:
misrep_jks.mu_b

In [ ]:
rerun_fi_vb=false
    # Compute Equity Finite Differences Method
    sfdf, rfdf = JointEq.compute_joint_eq_vb_results(fisf, rf, misrep_jks;
                                             rerun_fi_vb=rerun_fi_vb,
                                             fi_st_vb=jks.fi_st_vb,
                                             fi_rt_vb=jks.fi_rt_vb)


In [ ]:
sfdf

In [ ]:
rfdf

In [ ]:
jks.vbl

In [ ]:
setfield!(misrep_jks, :vbl, NaN)

In [ ]:
jks3 = JointEq.update_jks(misrep_jks)



In [ ]:
jks.vbl

In [ ]:
opt_st_vb, opt_rt_vb = JointEq.interp_optimal_vbs(misrep_jks, sfdf)
st_vb, rt_vb = JointEq.get_type_contingent_vbs(opt_st_vb,
                                           misrep_jks.fi_st_vb,
                                           misrep_jks.fi_rt_vb; 
                                           sf_defaults_first=true)
s1 = JointEq.joint_eq_fd(sf, rf, misrep_jks, st_vb=st_vb, rt_vb=rt_vb)
st_vb, rt_vb = JointEq.get_type_contingent_vbs(opt_rt_vb,
                                           misrep_jks.fi_st_vb,
                                           misrep_jks.fi_rt_vb; 
                                           sf_defaults_first=true)
s2 = JointEq.joint_eq_fd(sf, rf, misrep_jks, st_vb=st_vb, rt_vb=rt_vb)

opt_st_vb, opt_rt_vb = JointEq.interp_optimal_vbs(misrep_jks, rfdf)

# vbl that sets E'_s(vbl) to zero
st_vb, rt_vb = JointEq.get_type_contingent_vbs(opt_st_vb,
                                       misrep_jks.fi_st_vb,
                                       misrep_jks.fi_rt_vb; 
                                       sf_defaults_first=false)   
r1 = JointEq.joint_eq_fd(sf, rf, misrep_jks; st_vb=st_vb, rt_vb=rt_vb)

# vbl that sets E'_r(vbl) to zero   
st_vb, rt_vb = JointEq.get_type_contingent_vbs(opt_rt_vb,
                                       misrep_jks.fi_st_vb,
                                       misrep_jks.fi_rt_vb; 
                                       sf_defaults_first=false)
r2 = JointEq.joint_eq_fd(sf, rf, misrep_jks; st_vb=st_vb, rt_vb=rt_vb)
if abs.(r2[isnan.(r2[:, :st_vb]), :eq_deriv][1]) > 1e-2
    r2 = JointEq.refine_contingent_vbs(sf, rf, misrep_jks, st_vb, rt_vb)
end


In [ ]:
df =  vcat([s1, s2, r1, r2]...)
df[!, :sf_defaults_first] .= vcat([fill(true, 4), fill(false, 4)]...)

cols1 = [:sf_defaults_first,
         :fi_vb, :st_vb, :rt_vb, :vb,
         :eq_deriv, :eq_min_val]
df[:, vcat(cols1, [x for x in names(df) if !(x in cols1)]...)]




In [ ]:
   # Get Candidates
    df = JointEq.compile_opt_vb_results(sf, rf, misrep_jks, sfdf, rfdf)

In [ ]:

    # Get Candidates
    df = compile_opt_vb_results(sf, rf, jks, sfdf, rfdf)

    # Filter to Back out optimal vb
    return filter_joint_vb_results(df)

In [ ]:
JointEq.filter_joint_vb_results(df)

In [ ]:
fi_st_vb = NaN
fi_rt_vb = NaN
sf_defaults_first=true
vbl=59.9
bc = JointEqStructs.BondContract(jks.m, jks.c, jks.p)
if isnan(fi_st_vb)
    fi_st_vb = FullInfoEq.find_full_info_vb(sf, bc, jks.mu_b)
end

if isnan(fi_rt_vb)
    fi_rt_vb = FullInfoEq.find_full_info_vb(rf, bc, jks.mu_b)
end

st_vb, rt_vb = JointEq.get_type_contingent_vbs(vbl, fi_st_vb, fi_rt_vb;
                                               sf_defaults_first=sf_defaults_first)

jks = JointEqStructs.update_jks(jks; 
                     fi_st_vb=fi_st_vb,
                     fi_rt_vb=fi_rt_vb,
                     st_vb=st_vb,
                     rt_vb=rt_vb,
                     vbl=vbl)

In [ ]:
 println(string("vbl: ", getfield(jks, :vbl)))
    # ###############################################
    println("  ")
    println("================================================================")

    
    # Equity Boundary Values ########################
    eq_Vmax, vtgrid = JointEq.joint_eq_get_Vmax_vgrid(sf, rf, jks, jks.vbl;
                                              vtN=1500)

    sf_eq_vbl, sf_eq_max = JointEq.joint_eq_get_boundary_values(sf, jks,
                                                        jks.st_vb,
                                                        jks.vbl, eq_Vmax)
    rf_eq_vbl, rf_eq_max = JointEq.joint_eq_get_boundary_values(rf, jks,
                                                        jks.rt_vb,
                                                        jks.vbl, eq_Vmax)
    # ###############################################

    # Store Parameters ############################## 
    fdp = JointEq.JointFDParams(sf_eq_vbl,
                        sf_eq_max,
                        rf_eq_vbl,
                        rf_eq_max)
     # ###############################################   



In [ ]:
function ep_constructor(jf;
                        run_misrep::Bool=false,
                        run_pool_eq::Bool=true,
                        run_sep_eq::Bool=true,                       
                        sf_obj_fun::Symbol=:firm_value,
                        rf_obj_fun::Symbol=:MBR,
                        fi_fpath_name::String="",
                        rerun_full_info::Bool=true,
                        rerun_pool::Bool=true,
                        rerun_sep::Bool=true,
                        lb::Float64=.75,
                        ub::Float64=1.25,
                        mu_bN::Int64=20,
                        mu_bN2::Int64=10^5,
                        spline_k::Int64=3,
                        spline_bc::String="extrapolate")

    # Full Information Equilibrium
    fidf, fieqdf = FullInfoEq.get_fi_eq(jfep)
    
    # Extract Firms and set joint capital structure
    st, rt, ep_jks = JointEqStructs.form_firms_jks(jfep, fieqdf)
    
    # Check for missing parameters
    if any([isnan(getfield(ep_jks, x)) for x in [:mu_s, :m, :c, :p]])
        println("Missing Electronic Platform parameters")
        return
    end

    # Electronic Platform Full-Information Equilibrium ################
    if .&(isfile(fi_fpath_name), !rerun_full_info)
        fidf = CSV.read(fi_fpath_name)
                        #types=fidf_col_types)
        _, fieqdf = FullInfoEq.get_fi_eq(jfep, fidf=fidf)

        # Capture Full Information Optimal mu_b and MBR ###################
        fi_sf_mu_b = fieqdf[fieqdf[:, :type] .== :st, :mu_b][1]
        fi_rf_mu_b = fieqdf[fieqdf[:, :type] .== :rt, :mu_b][1]
        fi_sf_obj_val = fieqdf[fieqdf[:, :type] .== :st, sf_obj_fun][1]
        fi_rf_obj_val = fieqdf[fieqdf[:, :type] .== :st, sf_obj_fun][1]
        # #################################################################
    elseif rerun_full_info
        fidf, fieqdf = FullInfoEq.get_fi_eq(jfep)

        ep_sf_eqdf = find_optimal_bond_measure(ep_sf_svm; jks=ep_jks)
        ep_rf_eqdf = find_optimal_bond_measure(ep_rf_svm; jks=ep_jks)

        # Capture Full Information Optimal mu_b and MBR ###################
        fi_sf_mu_b = fieqdf[fieqdf[:, :type] .== :st, :mu_b][1]
        fi_rf_mu_b = fieqdf[fieqdf[:, :type] .== :rt, :mu_b][1]
        fi_sf_obj_val = fieqdf[fieqdf[:, :type] .== :st, sf_obj_fun][1]
        fi_rf_obj_val = fieqdf[fieqdf[:, :type] .== :st, sf_obj_fun][1]
        # #################################################################
    else
        fidf = DataFrame()
        fieqdf = DataFrame()

        fi_sf_mu_b = NaN
        fi_rf_mu_b =  NaN
        fi_sf_obj_val =  NaN
        fi_rf_obj_val =  NaN

        run_pool_eq = false
        run_sep_eq = false
    end
    # #################################################################

            
    # Electronic Platform Misrepresentation ###########################
    # Do risky firms have an incentive to copy the capital structure
    # of the safe firms?
    if run_misrep
        misrep_jks = deepcopy(ep_jks)
        setfield!(misrep_jks, :mu_s, 1.)
        if !isnan(fi_sf_mu_b)
            setfield!(misrep_jks, :mu_b, fi_sf_mu_b)
        end
        
        ep_misrep_eqdf = find_joint_optimal_vb(ep_jf, misrep_jks;
                                               mu_b=fi_sf_mu_b,
                                               rerun_fi_vb=true)

        # Add Objective Function columns
        ep_misrep_eqdf[!, :obj_fun] .= String(sf_obj_fun)
        ep_misrep_eqdf[2, :obj_fun] = "misrep"
        ep_misrep_eqdf[!, :eq_type] .= "misrep"
        
        # Reshape
        ep_misrep_eqdf = reshape_sf_rf_df(ep_misrep_eqdf)
    else
        ep_misrep_eqdf = DataFrame(mp)
    end
    # ##################################################################


    # Electronic Platform Pooling and Separating Equilibria ############
    eq_type = "all"
    run_pool_sep_eq = true
    if .&(run_pool_eq, !run_sep_eq)
        eq_type = "pooling"
    elseif .&(!run_pool_eq, run_sep_eq)
        eq_type = "separating"
    elseif .&(!run_pool_eq, !run_sep_eq)
        run_pool_sep_eq = false
    end

    ep_pool_eqdf = DataFrame()
    ep_sep_eqdf = DataFrame()
    if run_pool_sep_eq
        ep_eqdf = ep_pool_sep_eq(ep_jf, deepcopy(ep_jks),
                                 fi_sf_mu_b,
                                 fi_rf_mu_b,
                                 fi_rf_obj_val;
                                 equilibrium_type=eq_type,
                                 sf_obj_fun=sf_obj_fun,
                                 rf_obj_fun=rf_obj_fun,
                                 rerun=true,
                                 lb=lb, ub=ub,
                                 mu_bN=mu_bN, mu_bN2=mu_bN2,
                                 spline_k=spline_k,
                                 spline_bc=spline_bc)

        if ("pooling" in ep_eqdf[:, :eq_type])
            ep_pool_eqdf = ep_eqdf[ep_eqdf[:, :eq_type] .== "pooling", :]
        end
        if ("separating" in ep_eqdf[:, :eq_type])
            ep_sep_eqdf = ep_eqdf[ep_eqdf[:, :eq_type] .== "separating", :]
        end
    end
    # #################################################################


   # Form Electronic Platform Struct #################################
   return EPStruct(ep_jf, ep_sf_eqdf, ep_rf_eqdf,
                   ep_misrep_eqdf, ep_pool_eqdf, ep_sep_eqdf)
end



In [ ]:
function compute_misrep_eq(sf, rf, jks, fieqdf)
    
    
end

In [ ]:
function get_misrep_payoffs(jf, misrep_ft, fidf::DataFrame, fieqdf::DataFrame)
    copied_ft = misrep_ft == :st ? :rt : :st
    
    # Opt Capital Structure
    opt_mub = fieqdf[fieqdf[:, :type] .== :copied_ft, :mu_b][1]
    opt_bc = jf.bc
    
    for rmp in [:rm, :nrm]
        fr = getfield(getfield(jfep, misrep_ft), rmp).fr
        
        misrep_df = FullInfoEq.get_empty_fidf()
        if !isnothing(fr)
            # Find Optimal VB
            misrep_vb = FullInfoEq.find_full_info_vb(jfep.st.rm.fr, opt_bc, opt_mub)
    
            # Compute Misrepresentation Payoffs
            tmp = EqFinDiff.eq_fd(fr; vbl=misrep_vb, mu_b=opt_mub,
                                        m=opt_bc.m, c=opt_bc.c, p=opt_bc.p)

            tmp[!, :eq_type] .= :misrep
            tmp[!, :datetime] .= Dates.now()
            tmp[!, :type] .= misrep_ft
            tmp[!, :rmp] .= rmp
            tmp = vcat([misrep_df, tmp]...)
        end
        
        return misrep_df
    end
end


In [ ]:
# Safe Type Copies Risky Type #######

# Opt Capital Structure
rt_opt_mub = fieqdf[fieqdf[:, :type] .== :rt, :mu_b][1]
opt_bc = jfep.bc

# Find Optimal VB
st_misrep_vb = FullInfoEq.find_full_info_vb(jfep.st.rm.fr, opt_bc, rt_opt_mub)

# Compute Misrepresentation Payoffs
st_misrep_df = EqFinDiff.eq_fd(jfep.st.rm.fr; vbl=st_misrep_vb, mu_b=rt_opt_mub,
                m=opt_bc.m, c=opt_bc.c, p=opt_bc.p)


In [ ]:
st_misrep_df

In [ ]:
fieqdf

In [ ]:

# Electronic Platform Misrepresentation ###########################
# Do risky firms have an incentive to copy the capital structure
# of the safe firms?
if run_misrep
    misrep_jks = deepcopy(ep_jks)
    setfield!(misrep_jks, :mu_s, 1.)
    if !isnan(fi_sf_mu_b)
        setfield!(misrep_jks, :mu_b, fi_sf_mu_b)
    end

    ep_misrep_eqdf = find_joint_optimal_vb(ep_jf, misrep_jks;
                                           mu_b=fi_sf_mu_b,
                                           rerun_fi_vb=true)

    # Add Objective Function columns
    ep_misrep_eqdf[!, :obj_fun] .= String(sf_obj_fun)
    ep_misrep_eqdf[2, :obj_fun] = "misrep"
    ep_misrep_eqdf[!, :eq_type] .= "misrep"

    # Reshape
    ep_misrep_eqdf = reshape_sf_rf_df(ep_misrep_eqdf)
else
    ep_misrep_eqdf = DataFrame()
end
# ##################################################################



In [ ]:
tmp=FullInfoEq.find_optimal_bond_measure(jfep.st.rm.fr, jfep.bc)


In [ ]:
JointEq.get_jks_dir(jfep)

In [ ]:
JointEq.get_res_fpath(jfep)

In [ ]:
JointEq.get_jeq_res_fpath(jfep)

In [ ]:
JointEq.make_jeq_res_fpath(jfep)

In [ ]:
JointEq.get_jeq_jks_fpath(jfep)

In [ ]:
JointEq.get_jeq_common_params_dir(jfep)

In [ ]:
JointEq.make_res_fpath(jfep)

In [ ]:
jks_fpath = JointEq.make_jeq_jks_fpath(jfep)

In [ ]:
fi_fpath_name

In [ ]:
# #################################################################
fidf = CSV.read(fi_fpath_name)
                #types=fidf_col_types)

# Slice DataFrame
ep_sf_eqdf = DataFrame(fidf[1, :]) 
ep_rf_eqdf = DataFrame(fidf[2, :]) 

# Capture Full Information Optimal mu_b and MBR ###################
fi_sf_mu_b = ep_sf_eqdf[1, :mu_b]
fi_rf_mu_b = ep_rf_eqdf[1, :mu_b]
fi_sf_obj_val = ep_sf_eqdf[1, sf_obj_fun]
fi_rf_obj_val = ep_rf_eqdf[1, rf_obj_fun]
# #################################################################

In [ ]:
cond = .&(fidf[:, :type] .== :st, fidf[:, :rmp] .== :rm)
tmp = fidf[cond, :]
if !isempty(tmp)
    if isnan(tmp[:, :firm_value][1])
        tmp2 = FullInfoEq.find_optimal_bond_measure(getfield(jfep, :st).rm.fr; 
                                                    jks=ep_jks)
    end
end

In [ ]:
modls = ["Batch", "ModelObj", 
         "AnalyticFunctions", "BondPrInterp",
         "EqFinDiff", "FullInfoEq",
         "ModelPlots", "JointEq", "FullInfoMod"]
for modl in modls
    include(string(joinpath(module_path, modl), ".jl"))
end

In [ ]:
ep_jks = JointEq.store_ep_params(NaN; ep_m=1., ep_c=2., ep_p=3.)


In [ ]:
fidf3 = FullInfoMod.get_fi_results(jfep, fi_fpath_name)

In [ ]:
tmp = DataFrame()

In [ ]:
names(tmp)

In [ ]:
f(df) = .&(!isempty(df), :firm_value in names(df)) ? df[1, firm_value] : NaN

In [ ]:
function ep_constructor(jep, sf_bt, rf_bt;
                        ep_jks=JointKStruct(fill(NaN, 10)...),
                        ep_m::Float64=NaN,
                        ep_c::Float64=NaN,
                        ep_p::Float64=NaN,
                        run_misrep::Bool=false,
                        run_pool_eq::Bool=true,
                        run_sep_eq::Bool=true,                       
                        sf_obj_fun::Symbol=:firm_value,
                        rf_obj_fun::Symbol=:MBR,
                        fi_fpath_name::String="",
                        rerun_full_info::Bool=true,
                        rerun_pool::Bool=true,
                        rerun_sep::Bool=true,
                        lb::Float64=.75,
                        ub::Float64=1.25,
                        mu_bN::Int64=20,
                        mu_bN2::Int64=10^5,
                        spline_k::Int64=3,
                        spline_bc::String="extrapolate")


    # Measure of Firms and Standardized Bond
    ep_jks = store_ep_params(jep.mu_s;
                             ep_jks=ep_jks,
                             ep_m=ep_m,
                             ep_c=ep_c,
                             ep_p=ep_p)

    # Check for missing parameters
    if any([isnan(getfield(ep_jks, x)) for x in [:mu_s, :m, :c, :p]])
        println("Missing Electronic Platform parameters")
        return
    end

    # Adjust parameter dictionaries
    for var in [:alpha, :pi, :r, :gross_delta, :xi] #, :sigmal]
        sf_bt.mi._svm_dict[var] = getfield(jep.fcp, var)
        rf_bt.mi._svm_dict[var] = getfield(jep.fcp, var)
    end
    sf_bt.mi._svm_dict[:sigmal] = jep.sfp.sigmal
    rf_bt.mi._svm_dict[:sigmal] = jep.rfp.sigmal   

    # Form EP Safe Firm
    ep_sf_comb_num = get_batch_comb_num(sf_bt;
                                        iota=jep.sfp.iota,
                                        kappa=jep.kep,
                                        lambda=jep.sfp.lambda,
                                        sigmah=jep.sfp.sigmah)[1, :comb_num]
    _, ep_sf_svm = get_bt_mobj(;model=sf_bt.model, comb_num=ep_sf_comb_num)

    # Form EP Risky Firm
    ep_rf_comb_num = get_batch_comb_num(rf_bt;
                                        iota=jep.rfp.iota,
                                        kappa=jep.kep,
                                        lambda=jep.rfp.lambda,
                                        sigmah=jep.rfp.sigmah)[1, :comb_num]
    _, ep_rf_svm = get_bt_mobj(;model=rf_bt.model, comb_num=ep_rf_comb_num)



    # Joint Firm Constructor ##########################################
    ep_jf = joint_firm_constructor(ep_sf_svm, ep_rf_svm;
                                   jks=ep_jks,
                                   load_results_dfs=false)
    # #################################################################


    # Electronic Platform Full-Information Equilibrium ################
    if .&(isfile(fi_fpath_name), !rerun_full_info)
        fidf = CSV.read(fi_fpath_name)
                        #types=fidf_col_types)
        
        # Slice DataFrame
        ep_sf_eqdf = DataFrame(fidf[1, :]) 
        ep_rf_eqdf = DataFrame(fidf[2, :]) 

        # Capture Full Information Optimal mu_b and MBR ###################
        fi_sf_mu_b = ep_sf_eqdf[1, :mu_b]
        fi_rf_mu_b = ep_rf_eqdf[1, :mu_b]
        fi_sf_obj_val = ep_sf_eqdf[1, sf_obj_fun]
        fi_rf_obj_val = ep_rf_eqdf[1, rf_obj_fun]
        # #################################################################
    elseif rerun_full_info
        println(ep_sf_svm)
        println(ep_jks)

        ep_sf_eqdf = find_optimal_bond_measure(ep_sf_svm; jks=ep_jks)
        ep_rf_eqdf = find_optimal_bond_measure(ep_rf_svm; jks=ep_jks)

        # Capture Full Information Optimal mu_b and MBR ###################
        fi_sf_mu_b = ep_sf_eqdf[1, :mu_b]
        fi_rf_mu_b = ep_rf_eqdf[1, :mu_b]
        fi_sf_obj_val = ep_sf_eqdf[1, sf_obj_fun]
        fi_rf_obj_val = ep_rf_eqdf[1, rf_obj_fun]
        # #################################################################
    else
        ep_sf_eqdf = DataFrame()
        ep_rf_eqdf = DataFrame()

        fi_sf_mu_b = NaN
        fi_rf_mu_b =  NaN
        fi_sf_obj_val =  NaN
        fi_rf_obj_val =  NaN

        run_pool_eq = false
        run_sep_eq = false
    end
    # #################################################################

            
    # Electronic Platform Misrepresentation ###########################
    # Do risky firms have an incentive to copy the capital structure
    # of the safe firms?
    if run_misrep
        misrep_jks = deepcopy(ep_jks)
        setfield!(misrep_jks, :mu_s, 1.)
        if !isnan(fi_sf_mu_b)
            setfield!(misrep_jks, :mu_b, fi_sf_mu_b)
        end
        
        ep_misrep_eqdf = find_joint_optimal_vb(ep_jf, misrep_jks;
                                               mu_b=fi_sf_mu_b,
                                               rerun_fi_vb=true)

        # Add Objective Function columns
        ep_misrep_eqdf[!, :obj_fun] .= String(sf_obj_fun)
        ep_misrep_eqdf[2, :obj_fun] = "misrep"
        ep_misrep_eqdf[!, :eq_type] .= "misrep"
        
        # Reshape
        ep_misrep_eqdf = reshape_sf_rf_df(ep_misrep_eqdf)
    else
        ep_misrep_eqdf = DataFrame()
    end
    # ##################################################################


    # Electronic Platform Pooling and Separating Equilibria ############
    eq_type = "all"
    run_pool_sep_eq = true
    if .&(run_pool_eq, !run_sep_eq)
        eq_type = "pooling"
    elseif .&(!run_pool_eq, run_sep_eq)
        eq_type = "separating"
    elseif .&(!run_pool_eq, !run_sep_eq)
        run_pool_sep_eq = false
    end

    ep_pool_eqdf = DataFrame()
    ep_sep_eqdf = DataFrame()
    if run_pool_sep_eq
        ep_eqdf = ep_pool_sep_eq(ep_jf, deepcopy(ep_jks),
                                 fi_sf_mu_b,
                                 fi_rf_mu_b,
                                 fi_rf_obj_val;
                                 equilibrium_type=eq_type,
                                 sf_obj_fun=sf_obj_fun,
                                 rf_obj_fun=rf_obj_fun,
                                 rerun=true,
                                 lb=lb, ub=ub,
                                 mu_bN=mu_bN, mu_bN2=mu_bN2,
                                 spline_k=spline_k,
                                 spline_bc=spline_bc)

        if ("pooling" in ep_eqdf[:, :eq_type])
            ep_pool_eqdf = ep_eqdf[ep_eqdf[:, :eq_type] .== "pooling", :]
        end
        if ("separating" in ep_eqdf[:, :eq_type])
            ep_sep_eqdf = ep_eqdf[ep_eqdf[:, :eq_type] .== "separating", :]
        end
    end
    # #################################################################


   # Form Electronic Platform Struct #################################
   return EPStruct(ep_jf, ep_sf_eqdf, ep_rf_eqdf,
                   ep_misrep_eqdf, ep_pool_eqdf, ep_sep_eqdf)
end



In [ ]:
# INPUTS ###################################################
# Measure of Safe Firms
mu_s = .2

# Safe Firm's Risk Management Costs
st_rm_iota = 0.0 # 2.5 * 1e-4

# Transaction Costs and Volatility Risk Parameters
cvmdict = Dict{Symbol,Array{Float64,1}}(:sigmal => [0.15],
                                        :m  => [1.],
                                        :gross_delta => [0.02],
                                        :kappa  => [25 * 1e-4],
                                        :mu_b => [1.0],
                                        :xi => [1.0],
                                        :iota => [x for x in Batch.cvm_param_values_dict[:iota] 
                                                  if .&(x >= st_rm_iota, x <= 20. * 1e-4)])
svmdict = deepcopy(cvmdict)
svmdict[:lambda] = [.2]
svmdict[:iota] = [.0]
svmdict[:sigmah] = Batch.svm_param_values_dict[:sigmah]
# #########################################################
                
# Get Safe and Risky Firms' Full Info Optimal Results #####
firm_obj_fun = :firm_value 
cvmdf, svmdf, _ = ModelPlots.get_cvm_svm_dfs(cvmdict, svmdict;
                                             firm_obj_fun=firm_obj_fun)
# #########################################################
                

# Firm Types ##############################################
# Safe Type - RMP-Contingent Combinations
sf_rm_comb_num = cvmdf[abs.(cvmdf[:, :iota] .- st_rm_iota) .< 1e-5, :comb_num][1]
sf_nrm_comb_num = 0

# Risky Type - RMP-Contingent Combinations
rf_iota = cvmdict[:iota][5]
rf_lambda = svmdict[:lambda][1]
rf_sigmah = svmdict[:sigmah][2]
rf_rm_comb_num = cvmdf[abs.(cvmdf[:, :iota] .- rf_iota) .< 1e-5, :comb_num][1]
rf_nrm_comb_num = svmdf[.&(abs.(svmdf[:, :lambda] .- rf_lambda) .< 1e-5,
                           abs.(svmdf[:, :sigmah] .- rf_sigmah) .< 1e-5), :comb_num][1]
                
# Form Types
st = JointEq.firm_type_constructor(; cvm_comb_num=sf_rm_comb_num,
                                   svm_comb_num=sf_nrm_comb_num,
                                   set_rmpc_opt_k_struct=true,
                                   cvmdf=cvmdf, svmdf=svmdf)

rt = JointEq.firm_type_constructor(; cvm_comb_num=rf_rm_comb_num,
                                   svm_comb_num=rf_nrm_comb_num,
                                   set_rmpc_opt_k_struct=true,
                                   cvmdf=cvmdf, svmdf=svmdf)
# #########################################################
                
# #########################################################
# Optimal p/c Ratio: sf.optKS.p/sf.optKS.c ~~ 12
st_opt_rmp = JointEq.get_otc_fi_opt_rmp(st)
st_optKS = st_opt_rmp == :rm ? st.rm.fr.optKS : st.nrm.fr.optKS
ep_c = JointEq.round_value(st_optKS.c)
ep_ratio = JointEq.round_value(st_optKS.p/st_optKS.c)
ep_p = ep_ratio * ep_c
                
# # Set Capital Structure
# jks = JointEq.JointKStruct(mu_s, 
#                            st_optKS.mu_b,
#                            st_optKS.m, ep_c, ep_p)
                
 
# tcp = JointEq.get_types_common_params(st)
                
# # Joint Firm Object
# jf = JointEq.JointFirms(jks, st, rt, tcp,
#                         cvmdf, svmdf)
# # #########################################################
                
# # #########################################################


# # Make Directories and File Names ##########################
# # Make results directories


In [ ]:
getfield(jf.tcp, :xi)

In [ ]:
for modl in modls
    include(string(joinpath(module_path, modl), ".jl"))
end

In [ ]:
jks_fpath = JointEq.make_jeq_jks_fpath(jf)


In [ ]:
# Joint Firm Object
jf = JointEq.JointFirms(jks, st, rt,
                        cvmdf, svmdf)
# #########################################################


# Make Directories and File Names ##########################
# Make results directories
jks_fpath = JointEq.make_jeq_jks_fpath(jf)

# Full Information Equilibrium
fi_fpath_name = JointEq.get_jeq_contour_fname(jks_fpath, cn; eq_type="full_info")

# Misrepresentation
misrep_fpath_name = JointEq.get_jeq_contour_fname(jks_fpath, cn; eq_type="misrep")

# Pooling Equilibrium
pool_fpath_name = JointEq.get_jeq_contour_fname(jks_fpath, cn; eq_type="pooling")

# Separating Equilibrium
sep_fpath_name = JointEq.get_jeq_contour_fname(jks_fpath, cn; eq_type="separating")
# #########################################################


# Joint Equilibrium Parameters ############################
jep = JointEq.store_joint_eq_parameters(mu_s, sf.pm.kappa, sf.pm.kappa;
                                        s_iota=sf.pm.iota,
                                        s_lambda=sf.pm.lambda,
                                        s_sigmah=sf.pm.sigmah,
                                        r_iota=rf.pm.iota,
                                        r_lambda=rf.pm.lambda,
                                        r_sigmah=rf.pm.sigmah)
# #########################################################



In [ ]:


# Pick Capital Structure
jks = JointEq.JointKStruct(mu_s, 
                           sf.optKS.mu_b,
                           sf.optKS.m, ep_c, ep_p, 
                           NaN, NaN, NaN, NaN, NaN)

# Joint Firm Object
jf = JointEq.JointFirms(jks, sf, rf,
                        sf_bt,
                        rf_bt,  
                        cvmdf, svmdf)
# #########################################################



In [ ]:
st.rm.fr.optKS

In [ ]:
rt.nrm.fr.optKS